In [1]:
#imports

import pandapower as pp
import matplotlib as plt
import math

In [2]:
#the empty network where all the buses, trafos, lines will be added
net = pp.create_empty_network() 

#each house must be supplied with 4 kW of real power
ADMD=0.004 
#power factor of 0.9, we calculate the reactive power using the power triangle
q=0.004*math.tan(0.9)

#voltages along the transmission lines, there are 3: high, medium, low
HV=132 #high voltage, along the external grid transmission line
MV=11 #meduium voltage, along the transmission lines from the external grid to the local transformers in each road
LV=0.415 #low voltage, for transmission lines connected to the dwellings

In [3]:
#create buses
b0 = pp.create_bus(net, vn_kv = HV,  name="Power Source") #external grid
b1 = pp.create_bus(net, vn_kv = MV, geodata=(0,0), name="Medium Voltage") #after dropping to medium voltage

#worst case scenario:
#we will model for the worst case scenario, which are the dwellings furthest from the main transformer (group 1)
#the longer the transmission lines, the more are the losses along the lines, and hence higher power should be supplied
#after running the power flow for this case, the resulting power + power losses   will be multiplied by 3 and added as an extra load on bus 1 to represent the remaining 3 groups
#this is a conservative and safe estimate
#iterative process 

b2 = pp.create_bus(net, vn_kv = MV, name="b2") #medium voltage bus on one side of the transformer in group 1
b3 = pp.create_bus(net, vn_kv = LV, name="b3") #low voltage bus on other side of the transformer in group 1

#connection to group 1 of our houses
#boxes of houses (we need 4 of those, for group 1)
#each set of 11 dwellings represent a single bus in group 1
#they are all at low (nominal) voltage of 415 kV

b4 = pp.create_bus(net, vn_kv = LV, name="Set 1 in Group 1") #12
b5 = pp.create_bus(net, vn_kv = LV, name="Set 2 in Group 1") #12
b6 = pp.create_bus(net, vn_kv = LV, name="Set 3 in Group 1") #12
b7 = pp.create_bus(net, vn_kv = LV, name="Set 4 in Group 1") #12

net.bus

,name,vn_kv,type,zone,in_service
0,Power Source,132.000,b,None,True
1,Medium Voltage,11.000,b,None,True
2,b2,11.000,b,None,True
3,b3,0.415,b,None,True
4,Set 1 in Group 1,0.415,b,None,True
5,Set 2 in Group 1,0.415,b,None,True
6,Set 3 in Group 1,0.415,b,None,True
7,Set 4 in Group 1,0.415,b,None,True


In [4]:
#create ext grid
pp.create_ext_grid(net, bus=b0, vm_pu=1.04, name="Swing bus 0")

0

In [5]:
#create loads 
#each load represents 11 dwellings
#loads in group 1 are created on buses corresponding to the 4 sets

#no solar power scenario (worst case, and design criteria)
pp.create_load(net, bus=b4, p_mw=ADMD*12, q_mvar=q, name="Load 1, Set 1 in Group 1")
pp.create_load(net, bus=b5, p_mw=ADMD*12, q_mvar=q, name="Load 2, Set 2 in Group 1")
pp.create_load(net, bus=b6, p_mw=ADMD*12, q_mvar=q, name="Load 3, Set 3 in Group 1")
pp.create_load(net, bus=b7, p_mw=ADMD*13, q_mvar=q, name="Load 4, Set 4 in Group 1")

#solar power peak
#pp.create_load(net, bus=b4, p_mw=ADMD*12-0.0034*12, q_mvar=q, name="Load 1, Set 1 in Group 1")
#pp.create_load(net, bus=b5, p_mw=ADMD*12-0.0034*12, q_mvar=q, name="Load 2, Set 2 in Group 1")
#pp.create_load(net, bus=b6, p_mw=ADMD*12-0.0034*12, q_mvar=q, name="Load 3, Set 3 in Group 1")
#pp.create_load(net, bus=b7, p_mw=ADMD*13-0.0034*13, q_mvar=q, name="Load 4, Set 4 in Group 1")


#the load of rest of the development (remaining 118 dwellings) are added to bus 1
#this does not account for the losses in the lines
#after the first iteration is done, the power load will be edited and added to b1 instead of the dummy load

#trial 1
#pp.create_load(net, bus=b1, p_mw=ADMD*139, q_mvar=q, name="Dummy Load") #the rest of the houses in the development
#after running this, add power to the grid that represents the 3 other groups 

#trial 2
#add losses of 11.38% of the load
#pp.create_load(net, bus=b1, p_mw=(1+0.1138)*ADMD*139-0.0034*139, q_mvar=q, name="Remaining Branches") #the rest of the houses in the development
pp.create_load(net, bus=b1, p_mw=(1+0.1138)*ADMD*139, q_mvar=q, name="Remaining Branches") #the rest of the houses in the development


#display loads
net.load


,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,"Load 1, Set 1 in Group 1",4,0.048000,0.005041,0.0,0.0,NaN,1.0,True,wye
1,"Load 2, Set 2 in Group 1",5,0.048000,0.005041,0.0,0.0,NaN,1.0,True,wye
2,"Load 3, Set 3 in Group 1",6,0.048000,0.005041,0.0,0.0,NaN,1.0,True,wye
3,"Load 4, Set 4 in Group 1",7,0.052000,0.005041,0.0,0.0,NaN,1.0,True,wye
4,Remaining Branches,1,0.619273,0.005041,0.0,0.0,NaN,1.0,True,wye


In [6]:
#create transformers from parameters
#only two transformers are modeled in the simulation: HV to MV (main), and MV to LV (group 1)

#pfe is 100th of the rated value

pp.create_transformer_from_parameters(net, name="Trafo 0", hv_bus=b0, lv_bus=b1, sn_mva=1, vn_hv_kv=HV, vn_lv_kv=MV, vkr_percent=1, vk_percent=10, pfe_kw=1000/100, i0_percent=1/50)
pp.create_transformer_from_parameters(net, name="Trafo 1", hv_bus=b2, lv_bus=b3, sn_mva=0.25, vn_hv_kv=MV, vn_lv_kv=LV, vkr_percent=1, vk_percent=10, pfe_kw=250/100, i0_percent=1/50)

#display trafos
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service,pt_percent,oltc
0,Trafo 0,None,0,1,1.00,132.0,11.000,10.0,1.0,10.0,...,NaN,NaN,NaN,NaN,False,1,1.0,True,NaN,False
1,Trafo 1,None,2,3,0.25,11.0,0.415,10.0,1.0,2.5,...,NaN,NaN,NaN,NaN,False,1,1.0,True,NaN,False


In [7]:
#create lines

#20 km 132 kV line

#555 meters to get to the transformer in the middle 
#do we need to double the length? two cables?

#cable connected the main transformer and the transformer in group 1
#electrical characteristic from manufacturing datasheet

pp.create_line_from_parameters(net,from_bus=b1, to_bus=b2,length_km=0.655, r_ohm_per_km = 0.411, x_ohm_per_km = 0.120, c_nf_per_km = 320, max_i_ka=0.228)

#modeling longest low voltage cable length (furthest dwelling from trafo), assuming point load (conservative), to be more accurate, seperate loads and all cables

pp.create_line_from_parameters(net,from_bus=b3, to_bus=b4,length_km=0.380, r_ohm_per_km = 0.320, x_ohm_per_km = 0.073, c_nf_per_km = 430, max_i_ka=0.237)
pp.create_line_from_parameters(net,from_bus=b3, to_bus=b5,length_km=0.380, r_ohm_per_km = 0.320, x_ohm_per_km = 0.073, c_nf_per_km = 430, max_i_ka=0.237)
pp.create_line_from_parameters(net,from_bus=b3, to_bus=b6,length_km=0.355, r_ohm_per_km = 0.320, x_ohm_per_km = 0.073, c_nf_per_km = 430, max_i_ka=0.237)
pp.create_line_from_parameters(net,from_bus=b3, to_bus=b7,length_km=0.330, r_ohm_per_km = 0.320, x_ohm_per_km = 0.073, c_nf_per_km = 430, max_i_ka=0.237)

#display lines
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,None,None,1,2,0.655,0.411,0.120,320.0,0.0,0.228,1.0,1,None,True
1,None,None,3,4,0.380,0.320,0.073,430.0,0.0,0.237,1.0,1,None,True
2,None,None,3,5,0.380,0.320,0.073,430.0,0.0,0.237,1.0,1,None,True
3,None,None,3,6,0.355,0.320,0.073,430.0,0.0,0.237,1.0,1,None,True
4,None,None,3,7,0.330,0.320,0.073,430.0,0.0,0.237,1.0,1,None,True


In [8]:
#run power flow
pp.runpp(net, algorithm='gs')

In [9]:
#display results
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.040000,0.000000,-0.843748,-0.100597
1,1.025401,-4.432335,0.619273,0.005041
2,1.024930,-4.435531,0.000000,0.000000
3,1.005104,-8.911472,0.000000,0.000000
4,0.969293,-9.157023,0.048000,0.005041
5,0.969293,-9.157023,0.048000,0.005041
6,0.971734,-9.140282,0.048000,0.005041
7,0.971552,-9.157027,0.052000,0.005041


In [10]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.207235,0.029966,-0.207142,-0.038313,0.000093,-0.008346,0.010718,0.010788,0.010788,1.025401,-4.432335,1.024930,-4.435531,4.731405
1,0.049751,0.005431,-0.048000,-0.005041,0.001751,0.000391,0.069271,0.069272,0.069272,1.005104,-8.911472,0.969293,-9.157023,29.228765
2,0.049751,0.005431,-0.048000,-0.005041,0.001751,0.000391,0.069271,0.069272,0.069272,1.005104,-8.911472,0.969293,-9.157023,29.228765
3,0.049627,0.005404,-0.048000,-0.005041,0.001627,0.000363,0.069097,0.069098,0.069098,1.005104,-8.911472,0.971734,-9.140282,29.155357
4,0.053773,0.005438,-0.052000,-0.005041,0.001773,0.000397,0.074809,0.074810,0.074810,1.005104,-8.911472,0.971552,-9.157027,31.565393


In [11]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.843748,0.100597,-0.826508,-0.035007,0.017240,0.065590,0.003574,0.042344,1.04000,0.000000,1.025401,-4.432335,81.704244
1,0.207142,0.038313,-0.202901,-0.021704,0.004241,0.016609,0.010788,0.282446,1.02493,-4.435531,1.005104,-8.911472,82.212587


In [12]:
net.res_load

,p_mw,q_mvar
0,0.048000,0.005041
1,0.048000,0.005041
2,0.048000,0.005041
3,0.052000,0.005041
4,0.619273,0.005041


In [13]:
net

This pandapower network includes the following parameter tables:
   - bus (8 elements)
   - load (5 elements)
   - ext_grid (1 element)
   - line (5 elements)
   - trafo (2 elements)
   - bus_geodata (1 element)
 and the following results tables:
   - res_bus (8 elements)
   - res_line (5 elements)
   - res_trafo (2 elements)
   - res_ext_grid (1 element)
   - res_load (5 elements)

In [14]:
net.res_ext_grid

,p_mw,q_mvar
0,0.843748,0.100597
